In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf

from model import TransferModel

from keras.optimizers import adam_v2
from keras import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet_v2 import ResNet50V2
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout, Conv2D

2022-03-02 17:22:34.760367: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-02 17:22:34.760384: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Global settings
INPUT_DATA_DIR = 'color'
INPUT_SHAPE = (224, 224, 3)
TARGET_SIZE = (224, 224)
BATCH_SIZE = 32
TARGET = 'model'
BASE = 'ResNet'

In [3]:
train_datagen = ImageDataGenerator(rescale=1/255.)
train_data = train_datagen.flow_from_directory(directory=INPUT_DATA_DIR, batch_size=BATCH_SIZE, target_size=TARGET_SIZE, class_mode='categorical')

Found 71 images belonging to 3 classes.


In [4]:
base_model = ResNet50V2(include_top=False, input_shape=INPUT_SHAPE, weights=None)
base_model.trainable = True

2022-03-02 17:22:42.862615: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-02 17:22:42.862636: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-02 17:22:42.862651: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ilham): /proc/driver/nvidia/version does not exist
2022-03-02 17:22:42.862764: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
add_to_base = base_model.output
add_to_base = GlobalAveragePooling2D(data_format='channels_last', name='head_gap')(add_to_base)

In [6]:
new_output = Dense(len(train_data), activation='softmax', name='head_pred')(add_to_base)
com_model = Model(base_model.input, new_output)

In [7]:
com_model.compile(loss='categorical_crossentropy', optimizer=adam_v2.Adam(0.0001), metrics=['categorical_accuracy'])

In [8]:
history = com_model.fit(train_data, epochs=20, steps_per_epoch=(len(train_data)))

Epoch 1/20


2022-03-02 17:22:50.658782: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 102760448 exceeds 10% of free system memory.
2022-03-02 17:22:50.741506: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 106463232 exceeds 10% of free system memory.
2022-03-02 17:22:50.808714: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 102760448 exceeds 10% of free system memory.
2022-03-02 17:22:50.914383: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 102760448 exceeds 10% of free system memory.
2022-03-02 17:22:50.966562: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 102760448 exceeds 10% of free system memory.


3/3 [==============================] - 14s 3s/step - loss: 1.2402 - categorical_accuracy: 0.4225
Epoch 2/20
3/3 [==============================] - 12s 3s/step - loss: 0.9863 - categorical_accuracy: 0.5493
Epoch 3/20
3/3 [==============================] - 12s 3s/step - loss: 0.6650 - categorical_accuracy: 0.8592
Epoch 4/20
3/3 [==============================] - 12s 3s/step - loss: 0.5528 - categorical_accuracy: 0.9014
Epoch 5/20
3/3 [==============================] - 12s 3s/step - loss: 0.3996 - categorical_accuracy: 0.9437
Epoch 6/20
3/3 [==============================] - 12s 3s/step - loss: 0.2801 - categorical_accuracy: 0.9577
Epoch 7/20
3/3 [==============================] - 12s 3s/step - loss: 0.2132 - categorical_accuracy: 0.9577
Epoch 8/20
3/3 [==============================] - 12s 5s/step - loss: 0.1448 - categorical_accuracy: 0.9577
Epoch 9/20
3/3 [==============================] - 12s 5s/step - loss: 0.1127 - categorical_accuracy: 0.9859
Epoch 10/20
3/3 [======================

In [9]:
com_model.predict('test/94.png')

IndexError: list index out of range

In [17]:
from keras.preprocessing import image
import numpy as np

In [22]:
img = image.load_img('test/138.png', target_size=(224, 224))
img = image.img_to_array(img)
img /= 255.0
img = img.reshape(-1, *img.shape)
pred = com_model.predict(img)
classes = os.listdir('color')
pred[0]

array([0.47581592, 0.19572717, 0.32845694], dtype=float32)